In [1]:
%%capture
!pip install sentencepiece
!pip install transformers

In [2]:
#DO ALL NECESSARY IMPORTS
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import CamembertConfig, CamembertModel, CamembertTokenizer, CamembertTokenizer, CamembertForSequenceClassification
from transformers import BertModel, BertTokenizer
import torch
from sklearn.preprocessing import MinMaxScaler



In [3]:
#use bert embedding but using camembert as model
model_name = 'camembert-base'
camembert_model = CamembertModel.from_pretrained(model_name)
tokenizer = CamembertTokenizer.from_pretrained(model_name, revision="main", sentencepiece_model="/usr/local/lib/python3.10/dist-packages")


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

def get_bert_embedding(sentence, model, tokenizer):
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    return embeddings

def max_pool_embeddings(embeddings):
    return np.max(embeddings, axis=0)

def final_camembert(df):
    df['camembert_embedding'] =  df['sentence'].apply(lambda x: get_bert_embedding(x, camembert_model, tokenizer))
    df['cam_pooled_embedding'] = df['camembert_embedding'].apply(lambda x: max_pool_embeddings(x))
    return df

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/Data-Kaggle

/content/drive/MyDrive/Data-Kaggle


In [7]:
import pandas as pd

test = pd.read_csv('final_test.csv')
training = pd.read_csv('final_training.csv')

In [8]:
training.head(1)

,sentence,difficulty,encoded_diff,sentence_sp,tokens,tokens_no_stop,token_count_no_stop,token_count,nb_verbs,nb_nouns,nb_adj,nb_adv,words,tfidf_score_unigram,tfidf_score_bigram
0,Les coûts kilométriques réels peuvent diverger...,C1,4,Les coûts kilométriques réels peuvent diverger...,"['Les', 'coûts', 'kilométriques', 'réels', 'pe...","['coûts', 'kilométriques', 'réels', 'diverger'...",27,45,4,14,3,2,"['les', 'coûts', 'kilométriques', 'réels', 'pe...",27.043962,18.477813


In [9]:
training = final_camembert(training)

In [10]:
test = final_camembert(test)

In [12]:
type(training['sentence'][0])

str

In [19]:
pred_features = ['nb_verbs', 'nb_nouns','nb_adj','nb_adv' ]
#pred_features = []

In [20]:
X_cam = pd.DataFrame(training['cam_pooled_embedding'].tolist())
X = pd.concat([training[pred_features], X_cam], axis=1)
X.columns = X.columns.astype(str)
y = training[['encoded_diff']]
scaller = MinMaxScaler()
X = scaller.fit_transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [21]:
%%capture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

knn = KNeighborsClassifier()

k_range = list(range(1, 8))
param_grid = dict(n_neighbors=k_range)

# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', return_train_score=False,verbose=1)

knn_grid = grid.fit(X_train, y_train)


In [26]:
y_pred_knn = knn_grid.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_knn)
conf_matrix = confusion_matrix(y_test, y_pred_knn)
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test, y_pred_knn, average='weighted')
recall = recall_score(y_test, y_pred_knn, average='weighted')
f1 = f1_score(y_test, y_pred_knn, average='weighted')
print("Accuracy:", round(accuracy,2))
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

print("Precision:", round(precision, 2))
print("Recall:", round(recall, 2))
print("F1 Score:", round(f1, 2))

Accuracy: 0.17
Confusion Matrix:
 [[ 4 70  0  0 92  0]
 [ 4 79  0  0 75  0]
 [ 0 69  0  0 97  0]
 [ 1 81  0  0 71  0]
 [ 0 75  0  0 77  0]
 [ 3 74  0  0 88  0]]
Classification Report:
               precision    recall  f1-score   support

           0       0.20      0.10      0.14       166
           1       0.16      0.24      0.19       158
           2       0.12      0.02      0.03       166
           3       0.16      0.45      0.24       153
           4       0.16      0.17      0.17       152
           5       0.17      0.02      0.04       165

    accuracy                           0.16       960
   macro avg       0.16      0.17      0.13       960
weighted avg       0.16      0.16      0.13       960

Precision: 0.11
Recall: 0.17
F1 Score: 0.09


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
X_cam_pred = pd.DataFrame(test['cam_pooled_embedding'].tolist())
X_to_predict = pd.concat([test[pred_features], X_cam_pred], axis=1)
X_to_predict.columns = X_to_predict.columns.astype(str)
test['difficulty encoded'] = knn_grid.predict(X_to_predict)

In [ ]:
#function that cleans and return the final dataset to upload on kaggle
def prep_final_pred(df):
  number_to_level = {
    0: 'A1',
    1: 'A2',
    2: 'B1',
    3: 'B2',
    4: 'C1',
    5: 'C2'
}

  df['difficulty'] = df['difficulty encoded'].map(number_to_level)
  columns_to_keep = ['id', 'difficulty']
  final_df =  df[columns_to_keep]
  final_df.set_index('id', inplace=True)


  return final_df


In [ ]:
final_test = prep_final_pred(test)

In [ ]:
final_test.to_csv('submission.csv')

In [ ]:
final_test.head()

NameError: ignored

In [ ]:
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/Kaggle/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"


100% 8.30k/8.30k [00:02<00:00, 4.23kB/s]
Successfully submitted to Detecting the difficulty level of French texts